In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import torch
from random import choice
import math
import gym
import copy
import time
import shutil
import tensorflow as tf
import tensorflow.compat.v1 as tf
import time
from os import path
import gym
import numpy as np
import os

from matplotlib import animation, rc
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib as mpl

%matplotlib inline

In [2]:
path2 = 'C:\\Users\\raven\\MAP-Elites-NEAT'

In [3]:
os.chdir(path2)

## Create Graph files

In [4]:
def create_graphs(num_models,num_inp,num_out,dis_x,dis_y):
    #remove all files in Graph_folder
    if(os.getcwd()+'\\Pure_MAP' in [x[0] for x in os.walk(os.getcwd())]):
        shutil.rmtree(os.getcwd()+'\\Pure_MAP\\')

    #create new file
    os.mkdir(os.getcwd()+'\\Pure_MAP\\')

    #create number of files needed
    for x in range(dis_x):
        for y in range(dis_y):
            fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(x)+"_"+str(y)+".edgelist", "x")
            fh.close()
            fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(x)+"_"+str(y)+".txt", "x")
            fh.write(str(0.0))
            fh.close()

## Read model from file

In [5]:
def read_graph(x_val,y_val,num_inp,num_out):
    fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(x_val)+"_"+str(y_val)+".edgelist", "rb")
    G = nx.read_edgelist(fh,nodetype=int,create_using=nx.DiGraph(),data=(('weight',float),('activation',int),('inn_num',int)))
    fh.close() 
    
    #add input and output nodes
    for i in range(num_inp+num_out):
        G.add_node(i)
    return G

## Save Model

In [6]:
def write_graph(model,x_val,y_val,reward):
    nx.write_edgelist(model, os.getcwd()+"\\Pure_MAP\\model_"+str(x_val)+"_"+str(y_val)+".edgelist",data=['weight','activation','inn_num'])
    fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(x_val)+"_"+str(y_val)+".txt", "w")
    fh.write(str(reward))
    fh.close() 

## Mutate

In [7]:
def mutate(gene_model,num_models,max_inn_num,dis_x,dis_y):
    new_con_prob = .3
    con_to_node_prob = .3
    change_weights = .8
    
    if(random.random() < new_con_prob):
        add_con(gene_model,num_models,max_inn_num,dis_x,dis_y)
        
    if(random.random() < con_to_node_prob):
        con_to_node(gene_model,num_models,max_inn_num,dis_x,dis_y)
    
    if(random.random() < change_weights):
        new_weight(gene_model)    

## Add Connection

In [8]:
def add_con(model,num_models,max_inn_num,dis_x,dis_y):
    node_list = list(model.nodes)
    
    #get two random nodes
    length = len(node_list)
    node1 = node_list[random.randint(0,length-1)]
    node2 = node_list[random.randint(num_out,length-1)]
    
    #check if node2 is input node, node1 is an output node, or nodes are the same
    #if( (node2<num_inp) or (node1>=num_inp and node1<num_inp+num_out) or (node1 == node2)):
    #    return 0
    while( (node2<num_inp) or (node1>=num_inp and node1<num_inp+num_out) or (node1 == node2)):
        node1 = node_list[random.randint(0,length-1)]
        node2 = node_list[random.randint(num_out,length-1)]

    #check if adding edge will cause loop
    if(check_edges(model,node1,node2)):
        model.add_edge(node1,node2,weight=random.random(),activation=1,inn_num=inn_num((node1,node2),num_models,max_inn_num,dis_x,dis_y))
        max_inn_num[0]+=1

## Connection to Node

In [9]:
def con_to_node(model,num_models,max_inn_num,dis_x,dis_y):
    #if no connections, dont run rest of function
    temp = list(model.edges())
    if(len(temp)==0):
        return 0
    
    node1,node2 = temp[random.randint(0,len(temp)-1)]

    new_node = max(list(model.nodes))+1

    w1 = 1
    w2 = nx.get_edge_attributes(model,'weight').get((node1,node2))

    model.add_edge(node1,new_node,weight=w1,activation=1)
    model.add_edge(new_node,node2,weight=w2,activation=1)
    model.edges[node1,node2]['activation'] = 0
    
    model.edges[node1,new_node]['inn_num'] = inn_num((node1,new_node),num_models,max_inn_num,dis_x,dis_y)
    model.edges[new_node,node2]['inn_num'] = inn_num((new_node,node2),num_models,max_inn_num,dis_x,dis_y)

## Change weight

In [10]:
def new_weight(model):
    for edge in model.edges():
        if(random.random()<.1):
            #change weight
            weight = nx.get_edge_attributes(model,'weight').get(edge)
            model.edges[edge[0], edge[1]]['weight'] = random.random()
        else:
            #perturb weight
            weight = nx.get_edge_attributes(model,'weight').get(edge)
            model.edges[edge[0], edge[1]]['weight'] = model.edges[edge[0], edge[1]]['weight'] + random.random()*.2            

## Innovation numbers

In [11]:
def inn_num(edge,num_models,max_inn_num,dis_x,dis_y):
    #check if edge exists, if it does, set edge innovation number to match
    for x in range(dis_x):
        for y in range(dis_y):
            model = read_graph(x,y,num_inp,num_out)
            temp_dict = nx.get_edge_attributes(model,'inn_num')
            if edge in temp_dict.keys():
                return temp_dict[edge]
    
    #if new edge, then return current unused max innovation number
    max_inn_num[0]+=1
    return max_inn_num[0]-1

## Create TensorFlow graph

In [12]:
def get_tf_model(model,num_inp,num_out):
    #Create calc_dict of input nodes
    calc_dict = {}
    for i in range(num_inp): calc_dict[i]=tf.placeholder(tf.float32,[1])
        
    #get list of uncalculated nodes            
    temp_uncalc_node_list = []
    for i in model.nodes():
        if(i not in calc_dict):
            temp_uncalc_node_list.append(i)

    #calculate every node in uncalclated node list until list is empty
    index = -1
    while(len(temp_uncalc_node_list)>0):
        index+=1
        node = temp_uncalc_node_list[index]
        
        #if any input node is uncalculated, break, else, calculate current node
        temp_list = list(model.in_edges(node))
        temp_bool = False
        for i in temp_list:
            if i[0] not in calc_dict:
                temp_bool = True
                break
        if(temp_bool):
            continue

        #sum over all edge weight multiplied by the child nodes output
        tensor_sum = tf.zeros([1],dtype=tf.float32)
        for cur_edge in temp_list:
            weight = nx.get_edge_attributes(model,'weight').get(cur_edge)
            weight = tf.constant(weight,dtype=tf.float32)

            activation = nx.get_edge_attributes(model,'activation').get(cur_edge)
            activation = tf.constant(activation,dtype=tf.float32)

            node_output = calc_dict[cur_edge[0]]
            tensor_sum = node_output*weight*activation + tensor_sum

        #add calculated node to calc_dict and remove node from uncalc_node list
        calc_dict[node] = tf.math.tanh(tensor_sum)
        temp_uncalc_node_list.pop(index)
        index = -1
                
    #return list of output nodes
    out_list = []
    for i in range(num_inp,num_inp+num_out):
        if(i not in calc_dict):
            #not all output nodes are accounted for and are missing in calc_dict
            print("!!!!!!!!!!!!!!ERROR!!!!!!!!!!!!!!!!!")
        else:
            out_list.append(calc_dict[i])

    return out_list,calc_dict

## Get child model given parents

In [15]:
def get_child_gene_model(gene_model1,gene_model2):
    act_prob = .7
    child_model = nx.DiGraph()
    
    #add input and output nodes
    for i in range(num_inp+num_out):
        child_model.add_node(i)
    
    #get attributes of each parent graph
    weight_dict1 = nx.get_edge_attributes(gene_model1,'weight')
    weight_dict2 = nx.get_edge_attributes(gene_model2,'weight')
    act_dict1 = nx.get_edge_attributes(gene_model1,'activation')
    act_dict2 = nx.get_edge_attributes(gene_model2,'activation')
    inn_dict1 = nx.get_edge_attributes(gene_model1,'inn_num')
    inn_dict2 = nx.get_edge_attributes(gene_model2,'inn_num')
    
    #loop over first parent to see if nodes 
    for edge in weight_dict1:
        #if edge is in both models, 50% chance to choose either
        weight = 0
        activation = 1
        innovation = 0
        if(edge in weight_dict2):
            if(random.random() < .5):
                weight = weight_dict1.get(edge)
                activation = act_dict1.get(edge)
                innovation = inn_dict1.get(edge)
            else:
                weight = weight_dict2.get(edge)
                activation = act_dict2.get(edge)
                innovation = inn_dict2.get(edge)
        else:
            weight = weight_dict1.get(edge)
            activation = act_dict1.get(edge)
            innovation = inn_dict1.get(edge)
            
        #set activation to 1 with probability
        if(activation == 0):
            if(random.random()<act_prob):
                activation = 1
                
        #check if edge can be added without creating a circuit
        if(check_edges(child_model,edge[0],edge[1])):
            child_model.add_edge(edge[0],edge[1],weight=weight,activation=activation,inn_num=innovation)
        
    for edge in weight_dict2:
        #if edge is in both models, 50% chance to choose either
        if(edge not in weight_dict1):
            weight = 0
            activation = 1
            weight = weight_dict2.get(edge)
            activation = act_dict2.get(edge)
            innovation = inn_dict2.get(edge)
            
        #set activation to 1 with probability
        if(activation == 0):
            if(random.random()<act_prob):
                activation = 1
            
        #check if edge can be added without creating a circuit
        if(check_edges(child_model,edge[0],edge[1])):
            child_model.add_edge(edge[0],edge[1],weight=weight,activation=activation,inn_num=innovation)  
    return child_model

## Check for loops in graph

In [16]:
def check_edges(model,node1,node2):
    if((node2,node1) in model.edges() or (node1,node2) in model.edges()):
        return False
            
    temp_model = model.copy()
    temp_model.add_edge(node1,node2,weight=random.randrange(-2,2))
    
    try:
        nx.find_cycle(temp_model, orientation='original')
    except:
        return True
    return False

## Get existing graphs from MAP-Elites

In [17]:
'''def get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out):
    #get list of ones and zeros if graph exists
    available_graphs = []
    for i in range(dis_x):
        for j in range(dis_y):
            MAP_gene_model = read_graph(i,j,num_inp,num_out)
            if(not nx.is_empty(MAP_gene_model)):
                available_graphs.append((i,j))
         
    #if empty, add inddedx (0,0)
    if(len(available_graphs)==0):
        available_graphs.append((0,0))
    index_array = random.choices(available_graphs, k=num_models)
    return index_array'''

'def get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out):\n    #get list of ones and zeros if graph exists\n    available_graphs = []\n    for i in range(dis_x):\n        for j in range(dis_y):\n            MAP_gene_model = read_graph(i,j,num_inp,num_out)\n            if(not nx.is_empty(MAP_gene_model)):\n                available_graphs.append((i,j))\n         \n    #if empty, add inddedx (0,0)\n    if(len(available_graphs)==0):\n        available_graphs.append((0,0))\n    index_array = random.choices(available_graphs, k=num_models)\n    return index_array'

In [18]:
def get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out):
    
    #what if graphs got pulled from map elites weighted by how well they did?
    
    #get list of ones and zeros if graph exists
    available_graphs = []
    available_graph_rewards = []
    for i in range(dis_x):
        for j in range(dis_y):
            MAP_gene_model = read_graph(i,j,num_inp,num_out)
            if(not nx.is_empty(MAP_gene_model)):
                available_graphs.append((i,j))
                
                #get current graphs reward
                fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(i)+"_"+str(j)+".txt", "r")
                reward = float(fh.read())
                fh.close()
                available_graph_rewards.append(reward)
    
    #what if for every space not filled, an empty space is put in
    #if empty, add index (0,0)
    if(len(available_graphs)==0):
        available_graphs.append((0,0))
        available_graph_rewards.append(1.0)
        
    norm_avail_graph_r = [i/np.sum(np.array(available_graph_rewards)) for i in np.array(available_graph_rewards)]
    index_array = random.choices(available_graphs, weights=norm_avail_graph_r,k=num_models)
    return index_array

## Get MAP-Elites Space

In [19]:
dis_x = 100
dis_y = 100

## Hyper-parameters

In [20]:
num_inp = 24
num_out = 4

#num_inp = 4
num_bias = 10
num_inp = num_inp + num_bias
#num_out = 1

#changing num_models limits amount of saved models: fix this
num_models = 200
trials = 1
num_game_steps = 100
#max_inn_num = [num_inp+num_out]
max_inn_num = [0]
gens = 1

c1 = 3.0
c2 = 3.0
c3 = 3.0

generations = 1000
keep_amt = .3

crossover_chance = .5

## Run networks

In [21]:
#horizontal speed has index [2]
#legs touching groun has index [-11]
tf.reset_default_graph()
env = gym.make('BipedalWalker-v3')
create_graphs(num_models,num_inp,num_out,dis_x,dis_y)
with tf.Session() as sess:
    full_rew_list = []
    high_rew_list = []
    for gen in range(generations):
        total_a = time.time()

        #empty reward
        reward_list = []

        #get list of models to purturbate and test
        index_array = get_existing_graphs(num_models,dis_x,dis_y,num_inp,num_out)
        
        #run over all models
        for model_num in range(num_models):
            full_sum = 0
            #convert index number into 2d x,y values
            x,y = index_array[model_num]
            gene_model = read_graph(x,y,num_inp,num_out)
                
            print("model num: {} and {}".format(model_num,gene_model.edges()))
                
            #Mutation
            mutate(gene_model,num_models,max_inn_num,dis_x,dis_y)
            
            if(random.random()<crossover_chance):
                ind = random.randint(0,num_models-1)
                x,y = index_array[ind]
                
                gene_model1 = read_graph(x,y,num_inp,num_out)
                #mutate(gene_model1,num_models,max_inn_num,dis_x,dis_y)
                gene_model = get_child_gene_model(gene_model,gene_model1)
            
            if(nx.is_empty(gene_model)):
                reward_list.append((-38250.0,model_num))
                continue
            
            game_step_sum = -1
            g = tf.Graph()
            with g.as_default():
                with tf.Session() as sess:
                    #turn gene model into tensorflow model
                    tf_outputs,calc_dict = get_tf_model(gene_model,num_inp,num_out)

                    #heiristic initializations
                    avg_horizontal_speed = 0
                    perc_legs_on_ground = 0

                    
                    for trial in range(trials):
                        #get inputs
                        observation = env.reset()
                        observation = np.append(observation,[1])

                        #run model
                        for game_steps in range(num_game_steps):
                            game_step_sum+=1
                            #set up feed dictionary for model input
                            feed_dict = {}
                            for i in range(len(observation)): feed_dict[calc_dict[i]] = [observation[i]]
                            for i in range(len(observation),num_inp): feed_dict[calc_dict[i]] = [1.0]

                            actions = sess.run(tf_outputs,feed_dict=feed_dict)  #this is what costs the most time

                            action = np.array([actions[i][0] for i in range(num_out)])

                            #take action
                            observation, reward, done, info = env.step(action)
                            
                            #add observation to heiristics
                            avg_horizontal_speed += observation[2]
                            perc_legs_on_ground += observation[-11]

                            #append reward
                            full_sum+=reward
                            if(done):
                                env.close()
                                break
                                
            full_sum = avg_horizontal_speed*game_step_sum
            if(game_step_sum!=num_game_steps-1):
                full_sum-=40000
            reward_list.append((full_sum/trials,model_num))
            
            avg_horizontal_speed /= num_game_steps*trials
            perc_legs_on_ground /= num_game_steps*trials
            
            if(avg_horizontal_speed < 0):
                avg_horizontal_speed = 0
            
            x_val = int(avg_horizontal_speed*dis_x*5)
            y_val = int(perc_legs_on_ground*dis_y)
            if(y_val >= dis_y):
                y_val = dis_y-1
                
            if(x_val >= dis_x):
                x_val = dis_x-1
            
            #if current reward > old_reward, swap saved model
            fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(x_val)+"_"+str(y_val)+".txt", "r")
            old_reward = float(fh.read())
            fh.close() 
            #print("old_reward: {}".format(old_reward))
            
            MAP_gene_model = read_graph(x_val,y_val,num_inp,num_out)
            if(full_sum > old_reward or nx.is_empty(MAP_gene_model)):
                write_graph(gene_model,x_val,y_val,full_sum)
                print("{} > {} at {},{} and {} and {}".format(full_sum,old_reward,x_val,y_val,gene_model.edges(),MAP_gene_model.edges()))
                
        #calculate average reward
        sum_rew = 0
        for i in reward_list:
            sum_rew+= i[0]
        avg_rew = sum_rew/num_models
        #reward_list.append(avg_rew)
        full_rew_list.append(avg_rew)
        high_rew_list.append(max(reward_list))
        total_b = time.time()
        if(len(reward_list)!=0):
            print("Generation: {} Average reward: {} Time: {} Highest Reward: {}".format(gen,avg_rew,total_b-total_a,max(reward_list)))
        else:
            print("Generation: {} Average reward: {} Time: {} Highest Reward: {}".format(gen,avg_rew,total_b-total_a,"none"))
sess.close()
total_b = time.time()

C:\Users\raven\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


model num: 0 and []
model num: 1 and []
model num: 2 and []
1254.786559069902 > 0.0 at 63,75 and [(3, 36)] and []
model num: 3 and []
-39128.289778501836 > 0.0 at 64,42 and [(21, 37)] and []
model num: 4 and []
-39179.955337235515 > 0.0 at 63,40 and [(32, 35), (32, 38), (38, 35)] and []
model num: 5 and []
model num: 6 and []
model num: 7 and []
model num: 8 and []
model num: 9 and []
model num: 10 and []
model num: 11 and []
-40329.3857041914 > 0.0 at 0,16 and [(10, 34)] and []
model num: 12 and []
696.7342383988203 > 0.0 at 35,90 and [(8, 34), (8, 38), (38, 34)] and []
model num: 13 and []
-333.34248433694233 > 0.0 at 0,75 and [(25, 36)] and []
model num: 14 and []
904.6004132835569 > 0.0 at 45,82 and [(12, 35), (12, 38), (38, 35)] and []
model num: 15 and []
model num: 16 and []
model num: 17 and []
model num: 18 and []
model num: 19 and []
model num: 20 and []
-39160.633191411194 > 0.0 at 64,39 and [(17, 35), (17, 38), (38, 35)] and []
model num: 21 and []
-39022.4825165987 > 0.0 a

model num: 0 and [(11, 35), (11, 38), (38, 35)]
116.38327059144557 > 0.0 at 5,6 and [(11, 35), (11, 38), (11, 39), (17, 37), (17, 38), (38, 35), (38, 37), (39, 38)] and []
model num: 1 and [(5, 36)]
461.61570055581626 > 0.0 at 23,50 and [(5, 36), (5, 38), (28, 35), (28, 38), (38, 36), (38, 35)] and []
model num: 2 and [(10, 34)]
-40678.39158393741 > 0.0 at 0,30 and [(10, 34), (10, 36), (10, 38), (38, 34)] and []
model num: 3 and [(32, 35), (32, 38), (38, 35)]
121.35719783360251 > 0.0 at 6,67 and [(10, 36), (10, 38), (32, 35), (32, 38), (38, 35), (38, 36)] and []
model num: 4 and [(6, 35)]


KeyboardInterrupt: 

In [ ]:
x = [i for i in range(len(full_rew_list))]
y = full_rew_list
plt.plot(x,y)

In [ ]:
x = [i for i in range(len(high_rew_list))]
y = [i for i,j in high_rew_list]
plt.plot(x,y)

In [ ]:
high_rew_list[0]

In [ ]:
#Fixes some weird tensorflow error
tf.compat.v1.disable_eager_execution()
env = gym.make('BipedalWalker-v3')
#get model number
x = 75
y = 61

#get model from file
gene_model = read_graph(x,y,num_inp,num_out)
print(gene_model.edges())
#print(nx.get_edge_attributes(gene_model,'weight'))
#turn gene model into tensorflow model
tf_outputs,calc_dict = get_tf_model(gene_model,num_inp,num_out)


#get inputs
observation = env.reset()
observation = np.append(observation,[1])

#run model
rewards = 0
avg_horizontal_speed = 0
game_step_sum = 0
with tf.Session() as sess:
    for game_steps in range(500):#num_game_steps):
        game_step_sum+=1
        #set up feed dictionary for model input
        feed_dict = {}
        for i in range(len(observation)): feed_dict[calc_dict[i]] = [observation[i]]
        for i in range(len(observation),num_inp): feed_dict[calc_dict[i]] = [1.0]

        fin_out = []
        actions = sess.run(tf_outputs,feed_dict=feed_dict)  #this is what costs the most time

        for i in range(num_out):
            fin_out.append(actions[i][0])
        
        #take action
        env.render()
        action = np.array(fin_out)
        observation, reward, done, info = env.step(action)
        avg_horizontal_speed += observation[2]
        #print(reward)
        rewards+=reward
        if(done):
            break
env.close()
print(rewards)
print(avg_horizontal_speed*game_step_sum)

In [ ]:
data = np.random.rand(dis_x, dis_y)
#0 = red, 1 = blue

for i in range(dis_x):
    for j in range(dis_y):
        fh=open(os.getcwd()+"\\Pure_MAP\\model_"+str(i)+"_"+str(j)+".txt", "r")
        data[i][j] = fh.read()
        fh.close()
        MAP_gene_model = read_graph(i,j,num_inp,num_out)
        if(nx.is_empty(MAP_gene_model)):
            data[i][j] = -100000

#for i in range(dis_x):
#    print(data[i])
            
# create discrete colormap
cmap = mpl.cm.seismic
norm = mpl.colors.Normalize(vmin=-100000, vmax=100000)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(data, cmap=cmap, norm=norm)

plt.show()